# Main

> Module die alle functies die nodig zijn om de snelste route te vinden langs alle peilbuizen gegroepeerd per project.

In de huidige code gebruiken we de Openrouteservice API die publiekelijk gehost wordt. Deze heeft een aantal restricties. Zie https://openrouteservice.org/restrictions/.
De enige restrictie waar we in deze code last van hebben is het maximaal aantal locaties dat meegenomen kan worden in de optimalisatie. Volgens de documentatie is dat 50. Maar uit mijn testen blijkt het 70 te zijn.

Verder gebruiken we de Python client (https://openrouteservice-py.readthedocs.io/en/latest/) en construeren we dus niet zelf de curl calls.

In [ ]:
#| default_exp main

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import yaml
import pandas as pd
import logging
from datetime import datetime
from fastcore.utils import Path
from tqdm import tqdm

from project.data_get import get_data_from_azuresql, load_pickle
from project.route_get import create_optimized_route
from project.data_export import get_waypoint_coords, convert_routejson_to_df, create_route_prev_next, google_maps_route_url
from project.utils import get_project_root, make_filesystem_friendly, save_route_url, setup_logging

Get the settings from the settings.yaml file.

In [ ]:
#| export
project_root = get_project_root()

with open(project_root / 'settings.yaml', 'r') as f:
    settings = yaml.safe_load(f)

Get the data from the Azure SQL database in a pandas DataFrame or from a previously stored pickle file.

In [ ]:
#| export
use_pickle=True

if use_pickle:
    peilbuizen_df = load_pickle(settings['files']['pickle_file_input'])
else:
    peilbuizen_df = get_data_from_azuresql(sql_statement=settings['sql_statement']['peilbuizen'], 
                                       save_to_pickle=settings['files']['save_to_pickle'])

df_grouped = peilbuizen_df.groupby('project')

In [ ]:
#| hide
df_grouped.groups.keys()

dict_keys(['---', 'Bagger Dedemsvaart', 'Baggeren Beilervaart', 'Beekmaatregelen Reest', 'Boetelerveld_nw', 'Brongeb VL-AA', 'Dijken', 'Droogte Onderzoek NL fase 2 - bodemvochtmetingen', 'Droogtemeetnet WDOD', 'Ecologisch Effect Beekherstel Middenloop Vledder Aa Fase 1', 'Holtingerveld _Ootmaanlanden en Koningschut', 'Inrichting Dwingelderveld', 'Koekoekspolder grondwatermeetnet', 'Mastenbroek-IJssel', 'Meetnet De Wijk II', 'Nieuwveense landen', 'Nijstad Hoogeveen', 'Olde Maten en Veerslootlanden', 'Oldematen Reevediep', 'Oude Diep', 'Oude Diep Mantinge Bos_Zand', 'Oude Diep_Roode Brand', 'Oude Willem', 'Overijsselskanaal Deventer-Raalte', 'Paddenpol Zwolle-Olst', 'Primair meetnet blok 1', 'Primair meetnet blok 2', 'Primair meetnet blok 3', 'Randzone Ossenzijl Steenwijk', 'Reevediep', 'Reparatie meetpunten', 'Steenwijk_Kallenkote', 'Varsenerveld', 'Vecht', 'Vechterweerd', 'Vledder en Leierhooilanden', 'Wabos-KRW', 'ZUIDWOLDE-ZUID', 'Zandwetering Olst-Zuid_waterberging', 'dijken Stadsdi

In [ ]:
#| hide
# Test one project group
test_df_group = df_grouped.get_group('Primair meetnet blok 2')
test_name_group = 'Blok 2'

# Test random sample
df_random_test = peilbuizen_df.sample(n=100)

In [ ]:
#| hide
test_df_group.head()

,Id,Longitude,latitude,latest_measure_date,project
241,27GC901A;408995,6.180674,52.359436,2024-04-03 10:55:02,---
242,27HC007B;410283,6.202980,52.306282,2024-11-04 11:26:39,---
296,21FC011A;394286,6.207653,52.601814,2024-04-09 09:51:57,---
389,B21E0010;416303,6.181055,52.613954,2024-04-09 10:43:36,---
451,B21F0003;416597,6.219551,52.650246,2024-04-09 10:57:31,---


In [ ]:
#| hide
df_random_test.head()

,Id,Longitude,latitude,latest_measure_date,project
69,17AG505B;1326608,6.429767,52.880327,2024-09-23 11:24:03,None
485,17AG303C;1326553,6.398754,52.901672,2024-07-03 11:03:49,None
634,16DG903A;931512,6.050345,52.793872,2025-01-06 11:30:58,None
609,PB06-2;996646,5.866160,52.584272,2024-05-02 11:44:07,None
579,NPD-PB3D;40517,6.377796,52.776438,2024-08-30 08:10:28,Wabos-KRW


For every project in the peilbuizen dataframe, the shortest route is calculated and saved to an Excel file and a URL file.

In [ ]:
#| export

def create_group_route(start_address: str, # De startlocatie en eindlocatie opgegeven als adres
                       group_df: pd.DataFrame, # De pandas dataframe met peilbuizen in de groep, bijv. op basis van project
                       route_profile: str, # Het route profiel (transport methode) waarvoor de route berekend moet worden
                       project_name: str, # Naam van de groep, bijv. Project Vecht
                       output_dir: str = 'output', # Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder.
                       current_date: str = None, # De datum. Als niks wordt opgegeven, wordt de huidige datum gebruikt.
                       ) -> None:
    """
    Create optimized route for a group and save results
    """
    if current_date is None:
        current_date = datetime.now().strftime('%Y-%m-%d')
        
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    try:
        # Get optimized route
        route_json = create_optimized_route(start_address, group_df, route_profile)
        route_coords = get_waypoint_coords(route_json)
        
        # Create route dataframe
        optimized_route = convert_routejson_to_df(route_coords, group_df)
        route_table = create_route_prev_next(optimized_route)
        
        # Create filenames
        safe_project_name = make_filesystem_friendly(project_name)
        base_filename = f"peilbuizenroute_{safe_project_name}_{current_date}"
        
        # Save Excel file
        excel_path = output_path / f"{base_filename}.xlsx"
        route_table.to_excel(excel_path)
        
        # Save URL shortcut
        url = google_maps_route_url(route_coords)
        save_route_url(url, output_dir, f"{base_filename}.url")
        
    except Exception as e:
        print(f"Error processing group {project_name}: {str(e)}")

In [ ]:
#| hide
create_group_route(start_address=settings['calculation']['startlocation'],
                    group_df=test_df_group,
                    route_profile=settings['calculation']['distance_calculation_method'],
                    project_name=test_name_group,
                    output_dir=Path(settings['files']['path_results']) / 'test')


Error processing group Blok 2: 413 ({'code': 4, 'error': 'Too many locations ( 93 ) in query, maximum is set to 70'})


In [ ]:
#| export

def process_peilbuizen_routes(df: pd.DataFrame, # Dataframe van alle peilbuis groepen waarvoor route berekend wordt
                            start_address: str, # De startlocatie en eindlocatie opgegeven als adres
                            route_profile: str, # De route profiel (transport methode) waarvoor de route berekend moet worden
                            output_dir: str = 'output' # Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder.
                            ) -> None:
    """
    Process peilbuizen dataframe, create optimized routes for each group,
    and save results to Excel and URL files
    
    Args:
        df: DataFrame with peilbuizen data
        start_address: Starting location address
        route_profile: De route profiel (transport methode) waarvoor de route berekend moet worden
        output_dir: Directory to save output files
    """
    # Create output directory and setup logging
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    current_date = datetime.now().strftime('%Y-%m-%d')
    logger = setup_logging(output_dir, current_date)
        
    # Process each group
    grouped = df.groupby('project')
    total_groups = len(grouped)
    
    for project_name, group_df in tqdm(grouped, total=total_groups,desc="Processing peilbuis routes per project."):
        if len(group_df) < 4:
            msg = f"Project {project_name} heeft 3 of minder locaties. Optimale route uitrekenen is zinloos."
            print(msg)
            logger.info(msg)
        elif len(group_df) > 70:
            msg = f"Project {project_name} heeft meer dan 70 locaties. Optimale route uitrekenen is niet mogelijk op de publieke Openrouteservice. Oplossing: verdeel het project in kleinere delen, zet de Openrouteservice op eigen omgeving."
            print(msg)
            logger.info(msg)
        else:
            #TODO: Als deze functie een error returned, dus geen route vindt, komt dat niet in logfiles.
            try:
                create_group_route(
                    start_address=start_address,
                    group_df=group_df,
                    route_profile=route_profile,
                    project_name=project_name,
                    output_dir=output_dir,
                    current_date=current_date
                )
                logger.info(f"Successfully processed route for project: {project_name}")
            except Exception as e:
                error_msg = f"Error processing project {project_name}: {str(e)}"
                print(error_msg)
                logger.error(error_msg)

In [ ]:
#| export

process_peilbuizen_routes(df=peilbuizen_df,
                         start_address=settings['calculation']['startlocation'],
                         route_profile=settings['calculation']['distance_calculation_method'],
                         output_dir=settings['files']['path_results'])

Processing peilbuis routes per project.:   2%|▏         | 1/45 [00:00<00:37,  1.16it/s]2025-01-31 10:16:54:INFO:project.utils:Project Bagger Dedemsvaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-31 10:16:54:INFO:project.utils:Project Baggeren Beilervaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-31 10:16:54:INFO:project.utils:Project Beekmaatregelen Reest heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project Bagger Dedemsvaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Baggeren Beilervaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Beekmaatregelen Reest heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-01-31 10:16:55:INFO:project.utils:Successfully processed route for project: Boetelerveld_nw
Processing peilbuis routes per project.:  11%|█         | 5/45 [00:01<00:11,  3.54it/s]2025-01-31 10:16:55:INFO:project.utils:Project Brongeb VL-AA heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project Brongeb VL-AA heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-01-31 10:16:56:INFO:project.utils:Successfully processed route for project: Dijken
Processing peilbuis routes per project.:  16%|█▌        | 7/45 [00:02<00:15,  2.51it/s]2025-01-31 10:16:56:INFO:project.utils:Project Droogte Onderzoek NL fase 2 - bodemvochtmetingen heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-31 10:16:56:INFO:project.utils:Project Droogtemeetnet WDOD heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


...

2025-01-31 10:17:09:INFO:project.utils:Successfully processed route for project: Vledder en Leierhooilanden
Processing peilbuis routes per project.:  82%|████████▏ | 37/45 [00:17<00:04,  1.90it/s]2025-01-31 10:17:10:INFO:project.utils:Project ZUIDWOLDE-ZUID heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-31 10:17:10:INFO:project.utils:Project Zandwetering Olst-Zuid_waterberging heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-31 10:17:10:INFO:project.utils:Project dijken Stadsdijken Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-31 10:17:10:INFO:project.utils:Project effecten op WHH van gaswinning de Wijk II heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project ZUIDWOLDE-ZUID heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Zandwetering Olst-Zuid_waterberging heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project dijken Stadsdijken Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project effecten op WHH van gaswinning de Wijk II heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-01-31 10:17:11:INFO:project.utils:Successfully processed route for project: peilbesluit Kostverlorenzijl en Kloosterzijl
Processing peilbuis routes per project.:  93%|█████████▎| 42/45 [00:18<00:00,  3.06it/s]2025-01-31 10:17:11:INFO:project.utils:Project peilbesluit de Kruimels heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-31 10:17:11:INFO:project.utils:Project stedelijk Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project peilbesluit de Kruimels heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project stedelijk Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-01-31 10:17:12:INFO:project.utils:Successfully processed route for project: zandwinplassen
Processing peilbuis routes per project.: 100%|██████████| 45/45 [00:19<00:00,  2.35it/s]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()